In [1]:
import joblib

# load the trained xgboost model
model_path = "models/xgboost_best_model.pkl"
model = joblib.load(model_path)

print("Model loaded successfully!")


Model loaded successfully!


C:\Users\graha\AppData\Local\Programs\Python\Python310\lib\pickle.py:1718: UserWarning: [20:55:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\data\../common/error_msg.h:82: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [2]:
import requests

def fetch_live_weather():
    """Fetch real-time weather data for Cape Canaveral using Open-Meteo API."""
    API_URL = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": 28.3922,  # cape canaveral latitude
        "longitude": -80.6077,  # cape canaveral longitude
        "current": ["temperature_2m", "relative_humidity_2m", "precipitation",
                    "windspeed_10m", "windgusts_10m", "winddirection_10m",
                    "cloudcover", "pressure_msl"]
    }
    
    response = requests.get(API_URL, params=params)
    
    if response.status_code == 200:
        weather_data = response.json()["current"]
        
        # convert temperature from celsius to fahrenheit
        weather_data["temperature_2m"] = (weather_data["temperature_2m"] * 9/5) + 32
        
        print("Live weather data fetched successfully!")
        return weather_data
    else:
        print("Failed to fetch live data")
        return None

# test the function
live_weather = fetch_live_weather()
print(live_weather)


Live weather data fetched successfully!
{'time': '2025-03-22T00:45', 'interval': 900, 'temperature_2m': 56.3, 'relative_humidity_2m': 57, 'precipitation': 0.0, 'windspeed_10m': 9.6, 'windgusts_10m': 12.6, 'winddirection_10m': 34, 'cloudcover': 0, 'pressure_msl': 1020.9}


In [3]:
def get_manual_weather():
    """Allow users to manually input weather conditions in Fahrenheit."""
    print("\nEnter weather conditions manually:")

    weather_input = {
        "temperature_2m": float(input("Enter temperature (°F): ")),
        "relative_humidity_2m": float(input("Enter humidity (%): ")),
        "precipitation": float(input("Enter precipitation (mm): ")),
        "windspeed_10m": float(input("Enter wind speed (m/s): ")),
        "windgusts_10m": float(input("Enter wind gusts (m/s): ")),
        "winddirection_10m": float(input("Enter wind direction (°): ")),
        "cloudcover": float(input("Enter cloud cover (%): ")),
        "pressure_msl": float(input("Enter pressure (hPa): "))
    }

    return weather_input

# test manual input
manual_weather = get_manual_weather()
print(manual_weather)



Enter weather conditions manually:


Enter temperature (°F):  80
Enter humidity (%):  50
Enter precipitation (mm):  5
Enter wind speed (m/s):  20
Enter wind gusts (m/s):  30
Enter wind direction (°):  270
Enter cloud cover (%):  30
Enter pressure (hPa):  40


{'temperature_2m': 80.0, 'relative_humidity_2m': 50.0, 'precipitation': 5.0, 'windspeed_10m': 20.0, 'windgusts_10m': 30.0, 'winddirection_10m': 270.0, 'cloudcover': 30.0, 'pressure_msl': 40.0}


In [4]:
import pandas as pd

def predict_launch(weather_data):
    """Predict whether a rocket launch should proceed."""
    # define feature order (must match training data)
    feature_order = ["temperature_2m", "relative_humidity_2m", "precipitation",
                     "windspeed_10m", "windgusts_10m", "winddirection_10m",
                     "cloudcover", "pressure_msl"]
    
    # convert input into a dataframe
    weather_df = pd.DataFrame([weather_data], columns=feature_order)

    # make prediction
    prediction = model.predict(weather_df)[0]
    confidence = model.predict_proba(weather_df)[0][prediction] * 100

    result = "GO" if prediction == 1 else "NO-GO"
    print(f"**Launch Decision:** {result}")
    print(f"**Confidence Score:** {confidence:.2f}%")
    
    return result, confidence

# example prediction
predict_launch(manual_weather)


**Launch Decision:** GO
**Confidence Score:** 96.05%


('GO', 96.05005979537964)

In [ ]:
def main():
    """Main function to allow user interaction."""
    print("**Launch Prediction System**")
    choice = input("\nDo you want to fetch live weather data? (yes/no): ").strip().lower()

    if choice == "yes":
        weather_data = fetch_live_weather()
    else:
        weather_data = get_manual_weather()
    
    if weather_data:
        predict_launch(weather_data)
    else:
        print("No weather data available for prediction.")

# Run the user interaction
main()


**Launch Prediction System**



Do you want to fetch live weather data? (yes/no):  no



Enter weather conditions manually:


Enter temperature (°F):  200
Enter humidity (%):  200
